In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\kannapr1\Downloads\Insurance_Multilingual_Policy_Dataset.csv")

In [3]:
df.head()

,Policy_ID,Policy_Text_EN,Summarized_Text
0,1bddf126-1781-42f1-bdef-6d19e06906c6,This automobile insurance policy provides comp...,Comprehensive auto insurance covering accident...
1,18b33acc-1f39-4d9e-b045-42b7c75e830b,This home insurance policy covers damages to r...,"Home insurance covering structural damage, the..."
2,7c028259-8eb0-405f-a79b-4e004b85ebfc,This life insurance policy ensures financial s...,Life insurance ensuring financial support for ...
3,3184c4ec-41c6-48e3-9bc3-417a624e8b96,This automobile insurance policy provides comp...,Comprehensive auto insurance covering accident...
4,9b61707d-2378-4e15-a184-6a5dca03db6f,This health insurance policy offers extensive ...,Extensive health insurance including hospitali...


In [4]:
#pip install contractions

In [5]:
import contractions
import re

In [6]:
def normalize_text(text):

    if not isinstance(text, str):
        return text  
    #Convert to lower"
    text = text.lower() 

    #Expand contractions
    text = contractions.fix(text) 

    #Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  
    
    #Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()  
    
    return text

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Policy_ID        200 non-null    object
 1   Policy_Text_EN   200 non-null    object
 2   Summarized_Text  200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [8]:
df['Policy_text_Nomalized_EN'] = df['Policy_Text_EN'].apply(normalize_text)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Policy_ID                 200 non-null    object
 1   Policy_Text_EN            200 non-null    object
 2   Summarized_Text           200 non-null    object
 3   Policy_text_Nomalized_EN  200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [10]:
df.head()

,Policy_ID,Policy_Text_EN,Summarized_Text,Policy_text_Nomalized_EN
0,1bddf126-1781-42f1-bdef-6d19e06906c6,This automobile insurance policy provides comp...,Comprehensive auto insurance covering accident...,this automobile insurance policy provides comp...
1,18b33acc-1f39-4d9e-b045-42b7c75e830b,This home insurance policy covers damages to r...,"Home insurance covering structural damage, the...",this home insurance policy covers damages to r...
2,7c028259-8eb0-405f-a79b-4e004b85ebfc,This life insurance policy ensures financial s...,Life insurance ensuring financial support for ...,this life insurance policy ensures financial s...
3,3184c4ec-41c6-48e3-9bc3-417a624e8b96,This automobile insurance policy provides comp...,Comprehensive auto insurance covering accident...,this automobile insurance policy provides comp...
4,9b61707d-2378-4e15-a184-6a5dca03db6f,This health insurance policy offers extensive ...,Extensive health insurance including hospitali...,this health insurance policy offers extensive ...


In [11]:
#pip install tokenizers

In [12]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from transformers import AutoTokenizer

In [13]:
#pip install transformers

In [14]:
tokenizer = Tokenizer(models.BPE())

In [15]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [16]:
trainer = trainers.BpeTrainer()

In [17]:
tokenizer.train_from_iterator(df['Policy_text_Nomalized_EN'].dropna().tolist(), trainer)

In [18]:
tokenizer.save("trained_tokenizer.json")

In [19]:
# Tokenize text and store tokenized output
df["Tokenized_Policy_Text"] = df['Policy_text_Nomalized_EN'].apply(lambda text: tokenizer.encode(text).tokens)

In [20]:
df.head(1)

,Policy_ID,Policy_Text_EN,Summarized_Text,Policy_text_Nomalized_EN,Tokenized_Policy_Text
0,1bddf126-1781-42f1-bdef-6d19e06906c6,This automobile insurance policy provides comp...,Comprehensive auto insurance covering accident...,this automobile insurance policy provides comp...,"[this, automobile, insurance, policy, provides..."


In [21]:
# Apply Hugging Face tokenizer for embeddings using pre-trained BERT tokenizer
hf_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [22]:
df["Policy_text_Embeddings"] = df['Policy_text_Nomalized_EN'].apply(lambda text: hf_tokenizer.encode(text, add_special_tokens=True))

In [47]:
df.head(1)

,Policy_ID,Policy_Text_EN,Summarized_Text,Policy_text_Nomalized_EN,Tokenized_Policy_Text,Policy_text_Embeddings
0,1bddf126-1781-42f1-bdef-6d19e06906c6,This automobile insurance policy provides comp...,Comprehensive auto insurance covering accident...,this automobile insurance policy provides comp...,"[this, automobile, insurance, policy, provides...","[101, 2023, 9935, 5427, 3343, 3640, 7721, 6325..."


In [61]:
pip install torch


   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.4/204.1 MB 11.6 MB/s eta 0:00:18
   ---------------------------------------- 0.8/204.1 MB 9.8 MB/s eta 0:00:21
   ---------------------------------------- 1.3/204.1 MB 11.5 MB/s eta 0:00:18
   ---------------------------------------- 1.7/204.1 MB 10.7 MB/s eta 0:00:19
   ---------------------------------------- 2.3/204.1 MB 11.0 MB/s eta 0:00:19
    --------------------------------------- 2.7/204.1 MB 10.6 MB/s eta 0:00:19
    --------------------------------------- 3.2/204.1 MB 11.2 MB/s eta 0:00:18
    --------------------------------------- 3.6/204.1 MB 11.4 MB/s eta 0:00:18
    --------------------------------------- 4.1/204.1 MB 10.9 MB/s eta 0:00:19
    --------------------------------------- 4.6/204.1 MB 10.9 MB/s eta 0:00:19
    --------------------------------------- 5.0/204.1 MB 10.5 MB/s eta 0:00:19
    --------------------------------------- 5.0/204.1 MB 10.

In [61]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, pipeline
import tensorflow
from transformers import MarianMTModel, MarianTokenizer, TFMarianMTModel
import torch

In [51]:
model_name_fr = "Helsinki-NLP/opus-mt-en-fr"
model_name_es = "Helsinki-NLP/opus-mt-en-es"

In [63]:
tokenizer_fr = MarianTokenizer.from_pretrained(model_name_fr, trust_remote_code=True)
model_fr = TFMarianMTModel.from_pretrained(model_name_fr, trust_remote_code=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /Helsinki-NLP/opus-mt-en-fr/c860b5d50a25f2c6d063886cf36720016387b0ee94e7d33487bad4037b3e9ff6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tf_model.h5%3B+filename%3D%22tf_model.h5%22%3B&Expires=1741432983&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTQzMjk4M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9IZWxzaW5raS1OTFAvb3B1cy1tdC1lbi1mci9jODYwYjVkNTBhMjVmMmM2ZDA2Mzg4NmNmMzY3MjAwMTYzODdiMGVlOTRlN2QzMzQ4N2JhZDQwMzdiM2U5ZmY2P3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=LJp2ggr3Tt9iHibXPVjoE4oowUh1qfMQr~JEZjLhpcktP~gLP3XGa6rPCdsnAne0gnyl4XFaged8Eroax2jLt~r6-Qet2t4W6MfYP3UM1lMcnZAi~U9f7VxYJoRp0JXASp3J0XKYcF8v30WHnioNgN20vB98tQWgG9eYVGmMHEipr4cMV8tCSin2mcLeYJsM1aUHU1Qq930I9kgwTMprqAj6c0I-vQ978craGtWofCXUiMmOs6~dzUBfvE9USSVRJiTXtclSy7AE3R6oRU-ffp0gWVaEGeog7gUnoS5NcVwWchc3kqsrHHgtXYVsONWSMwT0tDzuuWGzSn2kadu

In [55]:
pip install sacremoses

   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   ----- ---------------------------------- 122.9/897.5 kB 2.4 MB/s eta 0:00:01
   -------------- ------------------------- 317.4/897.5 kB 3.3 MB/s eta 0:00:01
   ---------------------------- ----------- 634.9/897.5 kB 4.4 MB/s eta 0:00:01
   ---------------------------------------  890.9/897.5 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 897.5/897.5 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
